In [83]:
# -*- coding: utf-8 -*-
import math
import jieba
import jieba.posseg as psg
from gensim import corpora, models
from jieba import analyse
import functools
 
def get_stopword_list():
	stop_word_path = './/stopwords/stopword.txt'
	stopword_list = [sw.replace('\n','') for sw in open(stop_word_path).readlines()]
	return stopword_list
 
# 分词方法
def seg_to_list(sentence, pos=False):
	if not pos:
		# 不进行词性标注的分词方法
		seg_list = jieba.cut(sentence)
	else:
		# 进行词性标注的分词方法
		seg_list = psg.cut(sentence)
	return seg_list
 
# 去除干扰词，根据pos判断是否过滤除名词外的其他词性，再判断词是否在停用词表中，长度是否大于等于2等。
def word_filter(seg_list, pos=False):
	stopword_list = get_stopword_list()
	filter_list = []
	# 根据pos参数选择是否词性过滤
	# 不进行词性过滤，则将词性都标记为n,表示全部保留
	for seg in seg_list:
		if not pos:
			word = seg
			flag = 'n'
		else:
			word = seg.word
			flag = seg.flag
		if not flag.startswith('n'):
			continue
		# 过滤高停用词表中的词，以及长度为<2的词
		if not word in stopword_list and len(word)>1:
			filter_list.append(word)
	return filter_list
 
 # 数据加载
 
def load_data(pos=False,corpus_path='/home/jinhanqi/tf_program/topic2essay_data/null.txt'):
 	doc_list = []
 	for line in open(corpus_path, 'r'):
 		content = line.strip()
 		seg_list = seg_to_list(content, pos)
 		filter_list = word_filter(seg_list, pos)
 		doc_list.append(filter_list)
 	return doc_list
 
# idf值统计方法
def train_idf(doc_list):
	idf_dic = {}
	# 总文档数
	tt_count = len(doc_list)
	# 每个词出现的文档数
	for doc in doc_list:
		for word in set(doc):
			idf_dic[word] = idf_dic.get(word, 0.0) + 1.0
 
	# 按公式转换为idf值，分母加1进行平滑处理
	for k, v in idf_dic.items():
		idf_dic[k] = math.log(tt_count/(1.0+v))
	# 对于没有在字典中的词，默认其尽在一个文档出现，得到默认idf值
	default_idf = math.log(tt_count/(1.0))
	return idf_dic, default_idf
 
# topK
def cmp(e1, e2):
	import numpy as np 
	res = np.sign(e1[1] - e2[1])
	if res != 0:
		return res 
	else:
		a = e1[0] + e2[0]
		b = e2[0] + e1[0]
		if a > b:
			return 1
		elif a == b:
			return 0
		else:
			return -1
 
# TF-IDF类
class TfIdf(object):
	# 训练好的idf字典，默认idf值，处理后的待提取文本，关键词数量
	def __init__(self, idf_dic, default_idf, word_list, keyword_num):
		self.word_list = word_list
		self.idf_dic, self.default_idf = idf_dic, default_idf
		self.tf_dic = self.get_tf_dic()
		self.keyword_num = keyword_num
	#统计tf值
	def get_tf_dic(self):
		tf_dic = {}
		for word in self.word_list:
			tf_dic[word] = tf_dic.get(word, 0.0) + 1.0
		tt_count = len(self.word_list)
		for k,v in tf_dic.items():
			tf_dic[k] = float(v)/tt_count
		return tf_dic
	# 按公式计算tf-idf
	def get_tfidf(self):
		tfidf_dic = {}
		for word in self.word_list:
			idf = self.idf_dic.get(word, self.default_idf)
			tf = self.tf_dic.get(word, 0)
			tfidf = tf*idf
			tfidf_dic[word] = tfidf
		# 根据tf-idf排序，取排名前keyword_num的词作为关键词
		for k,v in sorted(tfidf_dic.items(), key=functools.cmp_to_key(cmp), reverse=True)[:self.keyword_num]:
			print(k)
		print ''
 
# 主题模型
class TopicModel(object):
	# 
	def __init__(self, doc_list, keyword_num, model="LSI", num_topics=4):
		# 使用gensim接口，将文本转为向量化表示
		self.dictionary = corpora.Dictionary(doc_list)
		# 使用BOW模型向量化
		corpus = [self.dictionary.doc2bow(doc) for doc in doc_list]
		# 对每个词，根据tf-idf进行加权，得到加权后的向量表示
		self.tfidf_model = models.TfidfModel(corpus)
		self.corpus_tfidf = self.tfidf_model[corpus]
		
		self.keyword_num = keyword_num
		self.num_topics = num_topics
		# 选择加载的模型
		if model == 'LSI':
			self.model = self.train_lsi()
		else:
			self.model = self.train_lda()
		# 得到数据集的主题-词分布
		word_dic = self.word_dictionary(doc_list)
		self.wordtopic_dic = self.get_wordtopic(word_dic)
 
	def train_lsi(self):
		lsi = models.LsiModel(self.corpus_tfidf, id2word=self.dictionary, num_topics=self.num_topics)
		return lsi
 
	def train_lda(self):
		lda = models.LdaModel(self.corpus_tfidf, id2word=self.dictionary, num_topics=self.num_topics)
		return lda
 
	def get_wordtopic(self, word_dic):
		wordtopic_dic = {}
		for word in word_dic:
			single_list = [word]
			wordcorpus = self.tfidf_model[self.dictionary.doc2bow(single_list)]
			wordtopic = self.model[wordcorpus]
			wordtopic_dic[word] = wordtopic
		return wordtopic_dic
 
    # 词空间构建方法和向量化方法，在没有gensim接口时的一般处理方法
	def word_dictionary(self, doc_list):
		dictionary = []
		for doc in doc_list:
			dictionary.extend(doc)
 
		dictionary = list(set(dictionary))
 
		return dictionary
 
	def doc2bowvec(self, word_list):
		vec_list = [1 if word in word_list else 0 for word in self.dictionary]
		return vec_list
 
	# 计算词的分布和文档的分布的相似度，取相似度最高的keyword_num个词作为关键词
	def get_simword(self, word_list):
		sentcorpus = self.tfidf_model[self.dictionary.doc2bow(word_list)]
		senttopic = self.model[sentcorpus]
		# 余弦相似度计算
		def calsim(l1, l2):
			a, b, c = 0.0, 0.0, 0.0
			for t1, t2 in zip(l1, l2):
				x1 = t1[1]
				x2 = t2[1]
				a += x1 * x1
				b += x1 * x1
				c += x2 * x2
			sim = a / math.sqrt(b * c) if not (b * c) == 0.0 else 0.0
			return sim
		# 计算输入文本和每个词的主题分布相似度
		sim_dic = {}
		for k,v in self.wordtopic_dic.items():
			if k not in word_list:
				continue
			sim = calsim(v, senttopic)
			sim_dic[k] = sim
 
		for k,v in sorted(sim_dic.items(), key=functools.cmp_to_key(cmp), reverse=True)[:self.keyword_num]:
			print(k)
		print ""
 
 
def tfidf_extract(word_list, pos=False, keyword_num=10):
	doc_list = load_data(pos)
	idf_dic, default_idf = train_idf(doc_list)
	tfidf_model = TfIdf(idf_dic, default_idf, word_list, keyword_num)
	tfidf_model.get_tfidf()
 
def textrank_extract(text, pos=False, keyword_num=10):
	textrank = analyse.textrank
	keywords = textrank(text, keyword_num)
	# 输出抽取出的关键词
	for keyword in keywords:
		print(keyword)
	#print()
 
def topic_extract(word_list, model, pos=False, keyword_num=10):
	doc_list = load_data(pos)
	topic_model = TopicModel(doc_list, keyword_num, model=model)
	topic_model.get_simword(word_list)


In [106]:
text = '赞 流水 ， 思 历史 人物 的 丰功伟绩 ； 瞧 那 奔腾 的 黄河 孕出 的 代代 天骄 ： 秦始皇 统一 中国 之 壮举 ； 唐玄宗 贞观之治 之 大貌 ； 毛泽东 解放 中国 之 雄壮 ； 历史 的 伟人 呀 ， 是 黄河 骄傲 的 子女 ， 是 潺潺流水 不息 的 生命 动力 ；'

In [60]:
text = '我 原本 也 是 自卑 者 。 如 一块 石子 ， 默默 的 躺 在 路边 ， 但 感谢 众多 博客 朋友 们 ， 让 我 重拾 了 自信 ， 远离 了 自卑 ， 让 我 明白 ： 即使 自己 是 一块 不起眼 的 石子 ， 也 可以 成为 一块 筑 基石 ， 为 繁忙 的 交通 奉献 了 自己 的 力量 。 李白 不是 曾 自信 的 说 ： “ 天生我才 必有用 ” 吗 ！ 我 也 自信 “ 长风破浪 会 有时 ， 直挂 云帆济 沧海 。 ” 因此 希望 所有 自卑 的 朋友 ― ― 告别 自卑 。'

In [101]:
text ='五 、 如果 您 是 位 特别 讲究 吃 穿 ， 追赶 时髦 的 人 ， 而 这 过分 的 追求 又 常常 弄 得 您 入不敷出 ， 故而 烦恼 不堪 。 那么 请 不要 沮丧 ， “ 卸下 负担 ” 培训班 将 教 您 学会 如何 玩钱 而 不 被 钱 玩弄 ， 如何 做到 收支 平衡 ， 还 您 一个 心情舒畅 的 自我 。 培训 时间 ： 一般 一个月 ， 可 根据 个人 情况 适当 缩短 或 延长 。'

In [102]:
print text

五 、 如果 您 是 位 特别 讲究 吃 穿 ， 追赶 时髦 的 人 ， 而 这 过分 的 追求 又 常常 弄 得 您 入不敷出 ， 故而 烦恼 不堪 。 那么 请 不要 沮丧 ， “ 卸下 负担 ” 培训班 将 教 您 学会 如何 玩钱 而 不 被 钱 玩弄 ， 如何 做到 收支 平衡 ， 还 您 一个 心情舒畅 的 自我 。 培训 时间 ： 一般 一个月 ， 可 根据 个人 情况 适当 缩短 或 延长 。


In [109]:
def getTFIDF(text):
    pos = True
    seg_list = seg_to_list(text, pos)
    filter_list = word_filter(seg_list, pos)
    return tfidf_extract(filter_list)

In [110]:
print('TF-IDF模型结果：')
getTFIDF(text)

TF-IDF模型结果：


/home/jinhanqi/.local/lib/python2.7/site-packages/ipykernel_launcher.py:40: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal


黄河
秦始皇
流水
毛泽东
子女
天骄
大貌
唐玄宗
历史
动力



In [98]:
textrank_extract(text)

中国
生命
历史
黄河
孕出
动力
人物
不息
伟人
奔腾


In [99]:
topic_extract(filter_list, 'LSI', pos)

/home/jinhanqi/.local/lib/python2.7/site-packages/ipykernel_launcher.py:40: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal


In [100]:
topic_extract(filter_list, 'LDA', pos)

/home/jinhanqi/.local/lib/python2.7/site-packages/ipykernel_launcher.py:40: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
